In [33]:
import ollama
import json
from langchain_community.document_loaders import UnstructuredPDFLoader
from langchain_experimental.llms.ollama_functions import OllamaFunctions
from langchain_core.pydantic_v1 import BaseModel, Field

In [34]:
# Load PDF and extract metadata
def load_pdf_and_metadata(pdf_path):
    loader = UnstructuredPDFLoader(pdf_path)
    data = loader.load()
    
    # Extract metadata
    metadata = data[0].metadata if data else {}
    
    # Extract text content
    content = data[0].page_content if data else ""
    
    return content, metadata

In [ ]:
# Main execution
pdf_path = "Summaries.pdf"
text, metadata = load_pdf_and_metadata(pdf_path)

In [ ]:
def segment_text(text, segment_size=3):
    paragraphs = text.split('\n\n')
    segments = []
    for i in range(0, len(paragraphs), segment_size):
        segment = ' '.join(paragraphs[i:i+segment_size])
        segments.append(segment)
    return segments

In [ ]:
segmented_text = segment_text(text)

In [ ]:
def sliding_window_segments(text, window_size=3, overlap=1):
    paragraphs = text
    segments = []
    i = 0
    while i < len(paragraphs):
        segment = ' '.join(paragraphs[i:i+window_size])
        segments.append(segment)
        i += (window_size - overlap)
    return segments


In [ ]:
better_segment = sliding_window_segments(segmented_text)

In [36]:
# Initialize the OllamaFunctions model
llm = OllamaFunctions(model="gemma2:2b", temperature=0.7, format="json")

In [41]:
def generate_question(segment):
    prompt = f"Generate a question based on the following text, focusing on key ideas related to culturally relevant pedagogy:\n\n{segment}"
    try:
        response = llm.invoke(prompt)
        return str(response).strip()
    except Exception as e:
        print(f"Error generating question: {e}")
        return None

def extract_answer(segment, question):
    prompt = f"Based on the following text, provide a concise answer to the question. Ensure the answer comes directly from the text:\n\nText: {segment}\n\nQuestion: {question}"
    try:
        response = llm.invoke(prompt)
        return str(response).strip()
    except Exception as e:
        print(f"Error extracting answer: {e}")
        return None

def generate_qa_pairs(segments):
    qa_pairs = []
    for segment in segments:
        question = generate_question(segment)
        if question is None:
            continue
        answer = extract_answer(segment, question)
        if answer is None:
            continue
        print("Raw Question:", question)
        print("Raw Answer:", answer)
        print("\n\n")
        qa_pairs.append({
            'segment': segment,
            'segment size': len(segment),
            'question': question,
            'answer': answer
        })
    return qa_pairs

In [ ]:
qa_pairs = generate_qa_pairs(better_segment)

# for i, pair in enumerate(qa_pairs, 1):
#     print(f"Pair {i}:")
#     print(f"Q: {pair['question']}")
#     print(f"A: {pair['answer']}")
#     print()

In [43]:
def save_qa_pairs(qa_pairs, output_file):
    with open(output_file, 'w') as f:
        json.dump(qa_pairs, f, indent=2)

# Example usage
save_qa_pairs(qa_pairs, 'final_dataset.json')

In [45]:
import json
import csv

def convert_json_to_csv(json_file, csv_file):
    # Read the JSON file
    with open(json_file, 'r', encoding='utf-8') as f:
        qa_pairs = json.load(f)

    # Open the CSV file in write mode with UTF-8 encoding
    with open(csv_file, 'w', newline='', encoding='utf-8') as f:
        # Extract the keys from the first item to create the header
        keys = qa_pairs[0].keys()
        writer = csv.DictWriter(f, fieldnames=keys)
        
        # Write the header row
        writer.writeheader()

        # Write each QA pair as a row in the CSV
        for qa_pair in qa_pairs:
            # Clean the question and answer fields
            qa_pair['question'] = qa_pair['question'].replace("content=", "").split("id=")[0].strip()
            qa_pair['answer'] = qa_pair['answer'].replace("content=", "").split("id=")[0].strip()
            
            writer.writerow(qa_pair)

# Example usage
convert_json_to_csv('final_dataset.json', 'final_dataset.csv')